# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Requirements" data-toc-modified-id="Requirements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Requirements</a></div><div class="lev1 toc-item"><a href="#KL-Functions" data-toc-modified-id="KL-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>KL Functions</a></div><div class="lev2 toc-item"><a href="#klGauss" data-toc-modified-id="klGauss-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>klGauss</a></div><div class="lev2 toc-item"><a href="#klBern" data-toc-modified-id="klBern-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>klBern</a></div><div class="lev1 toc-item"><a href="#Threshold-functions" data-toc-modified-id="Threshold-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Threshold functions</a></div><div class="lev2 toc-item"><a href="#Threshold-for-GLR-Bernoulli" data-toc-modified-id="Threshold-for-GLR-Bernoulli-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Threshold for GLR Bernoulli</a></div>

# Requirements

In [1]:
import numpy as np

import numba

In [2]:
import cython
%load_ext cython

# KL Functions

## klGauss

Generating some data:

In [3]:
def random_Gaussian(sig2=0.25):
    return sig2 * np.random.randn()

In [4]:
%timeit (random_Gaussian(), random_Gaussian())

1.12 µs ± 74.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In pure Python:

In [5]:
def klGauss(x: float, y: float, sig2=0.25) -> float:
    return (x - y)**2 / (2 * sig2**x)

In [6]:
%timeit klGauss(random_Gaussian(), random_Gaussian())

1.37 µs ± 54.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


With numba:

In [7]:
@numba.jit(nopython=True)
def klGauss_numba(x: float, y: float, sig2=0.25) -> float:
    return (x - y)**2 / (2 * sig2**x)

In [8]:
help(klGauss_numba)

Help on CPUDispatcher in module __main__:

klGauss_numba(x:float, y:float, sig2=0.25) -> float



In [9]:
%timeit klGauss_numba(random_Gaussian(), random_Gaussian())

The slowest run took 13.64 times longer than the fastest. This could mean that an intermediate result is being cached.
4.87 µs ± 7.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit klGauss_numba(random_Gaussian(), random_Gaussian())

1.43 µs ± 29.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
print(f"Speed up using Numba for klGauss was: {(1290-993)/(20300-993):.2g} faster!")

Speed up using Numba for klGauss was: 0.015 faster!


With Cython

In [12]:
%%cython --annotate

def klGauss_cython(double x, double y, double sig2=0.25) -> double:
    return (x - y)**2 / (2 * sig2**x)

In [13]:
help(klGauss_cython)

Help on built-in function klGauss_cython in module _cython_magic_49fb421fab37e1decb48fdb471e97e6b:

klGauss_cython(...)



In [14]:
%timeit klGauss_cython(random_Gaussian(), random_Gaussian())

1.21 µs ± 50.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
print(f"Speed up using Cython for klGauss was: {(1290-993)/(1100-993):.2g} faster!")

Speed up using Cython for klGauss was: 2.8 faster!


## klBern

Generating some data:

In [16]:
def random_Bern():
    return np.random.random()

In [17]:
%timeit (random_Bern(), random_Bern())

786 ns ± 30.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In pure Python:

In [18]:
from math import log

def klBern(x: float, y: float) -> float:
    x = max(1e-7, min(1 - 1e-7, x))
    x = max(1e-7, min(1 - 1e-7, x))
    return x * log(x/y) + (1-x) * log((1-x)/(1-y))

In [19]:
%timeit klBern(random_Bern(), random_Bern())

1.93 µs ± 96.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


With numba:

In [20]:
from math import log

@numba.jit(nopython=True)
def klBern_numba(x: float, y: float) -> float:
    x = max(1e-7, min(1 - 1e-7, x))
    x = max(1e-7, min(1 - 1e-7, x))
    return x * log(x/y) + (1-x) * log((1-x)/(1-y))

In [21]:
help(klBern_numba)

Help on CPUDispatcher in module __main__:

klBern_numba(x:float, y:float) -> float



In [22]:
%timeit klBern_numba(random_Bern(), random_Bern())

1.27 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [23]:
%timeit klBern_numba(random_Bern(), random_Bern())

1.14 µs ± 42.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [24]:
print(f"Speed up using Numba for klBern was: {(1740-753)/(996-753):.2g} faster!")

Speed up using Numba for klBern was: 4.1 faster!


With Cython

In [25]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [26]:
%%cython --annotate
from libc.math cimport log

def klBern_cython(double x, double y) -> double:
    x = max(1e-7, min(1 - 1e-7, x))
    x = max(1e-7, min(1 - 1e-7, x))
    return x * log(x/y) + (1-x) * log((1-x)/(1-y))

In [27]:
help(klBern_cython)

Help on built-in function klBern_cython in module _cython_magic_0ce0e64bf037172e33037b12661fd5c4:

klBern_cython(...)



In [28]:
%timeit klBern_cython(random_Bern(), random_Bern())

922 ns ± 36.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [29]:
print(f"Speed up using Cython for klBern was: {(1740-753)/(861-753):.2g} faster!")

Speed up using Cython for klBern was: 9.1 faster!


# Threshold functions

## Threshold for GLR Bernoulli

Generating some data:

In [30]:
def random_t0_s_t_delta(min_t: int=100, max_t: int=1000) -> (int, int, int, float):
    t0 = 0
    t = np.random.randint(min_t, max_t + 1)
    s = np.random.randint(t0, t)
    delta = np.random.choice([0.1, 0.05, 0.01, 0.005, 0.001, max(0.0005, 1/t)])
    return (t0, s, t, delta)

In [31]:
%timeit random_t0_s_t_delta()

7.04 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In pure Python:

In [32]:
def threshold(t0: int, s: int, t: int, delta: float) -> float:
    return np.log((s - t0 + 1) * (t - s) / delta)

In [33]:
%timeit threshold(*random_t0_s_t_delta())

10.2 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


It's *way* faster to use `math.log` instead of `numpy.log` (of course)!

In [34]:
from math import log

def threshold2(t0: int, s: int, t: int, delta: float) -> float:
    return log((s - t0 + 1) * (t - s) / delta)

In [35]:
%timeit threshold2(*random_t0_s_t_delta())

7.93 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In numba:

In [36]:
from math import log

@numba.jit(nopython=True)
def threshold_numba(t0: int, s: int, t: int, delta: float) -> float:
    return log((s - t0 + 1) * (t - s) / delta)

In [37]:
help(threshold_numba)

Help on CPUDispatcher in module __main__:

threshold_numba(t0:int, s:int, t:int, delta:float) -> float



In [38]:
%timeit threshold_numba(*random_t0_s_t_delta())

7.57 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [39]:
print(f"Speed up using Cython for thresold was: {(7510-7200)/(7750-7200):.2g} faster!")

Speed up using Cython for thresold was: 0.56 faster!


In Cython:

In [40]:
%%cython --annotate
from libc.math cimport log

cpdef double threshold_cython(int t0, int s, int t, double delta):
    return log((s - t0 + 1) * (t - s) / delta)

In [41]:
%%cython --annotate
from libc.math cimport log

def threshold_cython2(int t0, int s, int t, double delta) -> double:
    return log((s - t0 + 1) * (t - s) / delta)

In [42]:
help(threshold_cython)

Help on built-in function threshold_cython in module _cython_magic_4e48b0e25538ff2883ba5a92832943be:

threshold_cython(...)



In [43]:
%timeit threshold_cython(*random_t0_s_t_delta())

7.24 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [44]:
%timeit threshold_cython2(*random_t0_s_t_delta())

8.8 µs ± 994 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [45]:
print(f"Speed up using Cython for thresold was: {abs(7510-7200)/abs(7070-7200):.2g} faster!")

Speed up using Cython for thresold was: 2.4 faster!
